<a href="https://colab.research.google.com/github/jyoti-kumbhar/Data-Science/blob/main/prac4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Practical No.4
*   Aim: Hypothesis Testing
1. Formulate null and alternative hypotheses for a given problem.

2. Conduct a hypothesis test using appropriate statistical tests (e.g., t-test, chi-
square test).

3. Interpret the results and draw conclusions based on the test outcomes.
*  T091 Jyoti Kumbhar

**t Test**


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [ ]:
df = pd.read_csv("/content/Loan_default.csv")
df.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [ ]:
print(df.columns.tolist())

['LoanID', 'Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner', 'Default']


In [ ]:
df = pd.read_csv("/content/Loan_default.csv").dropna()

In [ ]:
# define null hypothesis
H0 = "The average loan interest rate is equal to 12%."
H1 = "The average loan interest rate is more than 12%."
# calculate the test statistic
t_stat, p_value_two_tailed = stats.ttest_1samp(df['InterestRate'],12)
# Convert two-tailed p-value to one-tailed (because your H1 is 'greater than')
p_value = p_value_two_tailed / 2
# Results
print("Test statistic:", t_stat)
print("One-tailed p-value:", p_value)
# Conclusion
if t_stat > 0 and p_value < 0.05:
    print("Reject the null hypothesis.")
else:
    print("Accept the null hypothesis.")

Test statistic: 113.66425722485945
One-tailed p-value: 0.0
Reject the null hypothesis.


Conclusion:

The results show a very high test statistic (t = 113.66) and a one-tailed p-value of 0.00. Since the p-value is well below the 0.05 significance level and the test statistic is positive, we reject the null hypothesis. This provides strong evidence that the average loan interest rate in the dataset is significantly higher than 12%.


**Two sampled T-test**

InterestRate vs Default:

Check if InterestRate differs between defaulters and non-defaulters.

In [ ]:
from scipy.stats import ttest_ind
# define null hypothesis
H0 = "There is no differnce in interest rate"
H1 = "There is differnce in interest rate"
# Ensure InterestRate is numeric
df['InterestRate'] = pd.to_numeric(df['InterestRate'], errors='coerce')
# Separate groups
default_group = df[df['Default'] == 1]['InterestRate'].dropna()
non_default_group = df[df['Default'] == 0]['InterestRate'].dropna()
# Check if groups have enough data
if len(default_group) < 2 or len(non_default_group) < 2:
    print("Not enough data to perform t-test")
else:
    # Compute means and standard deviations
    mean_default = default_group.mean()
    mean_non_default = non_default_group.mean()
    std_default = default_group.std()
    std_non_default = non_default_group.std()
    print("Default group InterestRate:\n", default_group.head())
    print("Non-default group InterestRate:\n", non_default_group.head())
    print(f"Mean InterestRate (Default = 1): {mean_default}")
    print(f"Mean InterestRate (Default = 0): {mean_non_default}")
    print(f"STD InterestRate (Default = 1): {std_default}")
    print(f"STD InterestRate (Default = 0): {std_non_default}")
    t_stat, p_val = ttest_ind(default_group, non_default_group, equal_var=False)  # Welch's t-test
    print(f"t-statistic: {t_stat}")
    print(f"p-value: {p_val}")
    # Decide on null hypothesis
    if p_val < 0.05:
        print("Reject the null hypothesis")
    else:
        print("Accept the null hypothesis")

Default group InterestRate:
 2     21.17
5     22.72
8     23.94
11     5.84
18    14.00
Name: InterestRate, dtype: float64
Non-default group InterestRate:
 0    15.23
1     4.81
3     7.07
4     6.51
6    19.11
Name: InterestRate, dtype: float64
Mean InterestRate (Default = 1): 15.89622668869929
Mean InterestRate (Default = 0): 13.176993717156861
STD InterestRate (Default = 1): 6.32030418882371
STD InterestRate (Default = 0): 6.612264902392917
t-statistic: 69.27344928560395
p-value: 0.0
Reject the null hypothesis


Conclusion:

The mean interest rate for the default group (≈15.90) is higher than the non-default group (≈13.18). The t-test shows a very large t-statistic and a p-value of 0, indicating a statistically significant difference. We reject the null hypothesis interest rates are significantly higher for borrowers who default.


**paired sample t test**

In [ ]:
H1 = "Significant difference exists between LoanAmount and Income"
H0 = "No significant difference exists between LoanAmount and Income"
# Drop rows with missing LoanAmount or Income
df = df[['LoanAmount', 'Income']].dropna()
scaled_loan = df['LoanAmount'] / df['LoanAmount'].max() * 100
scaled_income = df['Income'] / df['Income'].max() * 100
# Paired t-test
t_stat, pval = stats.ttest_rel(scaled_loan, scaled_income)
print("t-statistic:", t_stat)
print("p-value:", pval)
# Conclusion
if pval < 0.05:
    print("Reject null hypothesis")
else:
    print("Accept null hypothesis")

t-statistic: -52.140673090034106
p-value: 0.0
Reject null hypothesis


Conclusion:

The paired t-test shows a very large negative t-statistic and a p-value of 0, indicating a statistically significant difference between scaled LoanAmount and Income. We reject the null hypothesis — borrowers’ loan amounts are significantly different from their incomes, meaning most borrowers take loans that are not proportional to their income.

**Z test**

In [ ]:
from statsmodels.stats import weightstats as stests
H0 = 'The mean loan amount = 100,000'
H1 = 'The mean loan amount ≠ 100,000'
df = pd.read_csv("/content/Loan_default.csv")
df['LoanAmount'] = pd.to_numeric(df['LoanAmount'], errors='coerce')
loan_amounts = df['LoanAmount'].dropna()
# One-sample z-test against mean = 100000
ztest, pval = stests.ztest(loan_amounts, value=100000)
print("z-statistic:", ztest)
print("p-value:", float(pval))
# Conclusion
if pval < 0.05:
    print("Reject null hypothesis")
else:
    print("Accept null hypothesis")

z-statistic: 196.72470164669758
p-value: 0.0
Reject null hypothesis


Conclusion:

The one-sample z-test shows a very large z-statistic and a p-value of 0, indicating a statistically significant difference. We reject the null hypothesis — the mean loan amount is significantly different from 100,000.


**Two sample z test**

In [ ]:
H0 = 'The mean loan amount is not same for defaulted and non-defaulted borrowers'
H1 = 'The mean loan amount is same for defaulted and non-defaulted borrowers'
import pandas as pd
from statsmodels.stats import weightstats as stests
# Load dataset
df = pd.read_csv("/content/Loan_default.csv")
df.columns = df.columns.str.strip().str.lower()
df['loanamount'] = pd.to_numeric(df['loanamount'], errors='coerce')
# Separate groups
default_group = df[df['default'] == 1]['loanamount'].dropna()
nondefault_group = df[df['default'] == 0]['loanamount'].dropna()
# Two-sample z-test
zstat, pval = stests.ztest(default_group, nondefault_group)
print("z-statistic:", zstat)
print("p-value:", float(pval))
# Conclusion
if pval < 0.05:
    print("Reject null hypothesis")
else:
    print("Accept null hypothesis")

z-statistic: 43.95569282906432
p-value: 0.0
Reject null hypothesis


Conclusion:

The two-sample z-test shows a very large z-statistic and a p-value of 0, indicating a statistically significant difference. We reject the null hypothesis — the mean loan amount is significantly different between defaulted and non-defaulted borrowers.

**Chi Square test**

In [ ]:
H0='Education level and loan default are not dependent.'
H1='Education level and loan default are dependent.'
import pandas as pd
from scipy.stats import chi2_contingency
df = pd.read_csv("/content/Loan_default.csv")
df.columns = df.columns.str.strip().str.lower()
df['education'] = df['education'].astype(str)
df['default'] = df['default'].astype(str)
# Create contingency table
contingency_table = pd.crosstab(df['education'], df['default'])
print("Contingency Table:\n", contingency_table)
# Chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)
print("Chi-square statistic:", chi2)
print("p-value:", p)
# Conclusion
if p < 0.05:
    print("Reject null hypothesis")
else:
    print("Accept null hypothesis")

Contingency Table:
 default          0     1
education               
Bachelor's   56577  7789
High School  55673  8230
Master's     56633  6908
PhD          56811  6726
Chi-square statistic: 214.01898255592704
p-value: 3.9404864175508716e-46
Reject null hypothesis


Conclusion:

The Chi-square test shows a very large statistic and an extremely small p-value, indicating a statistically significant association. We reject the null hypothesis — education level and loan default are dependent; default rates vary by education level.